In [5]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import time
import random
import re
import os
import glob
from google.colab import drive

# Montar Google Drive (para Colab)
drive.mount('/content/drive')

def buscar_imdb(titulo_pelicula, año_pelicula):
    """
    Busca una película en IMDB por título y año
    """
    cabeceras = {
        'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36'
    }

    url_busqueda = f"https://www.imdb.com/find/?q={titulo_pelicula.replace(' ', '+')}&s=tt"

    print(f"Buscando en IMDB: {titulo_pelicula} ({año_pelicula})")

    try:
        respuesta = requests.get(url_busqueda, headers=cabeceras)
        if respuesta.status_code != 200:
            print(f"Error al buscar en IMDB: Código {respuesta.status_code}")
            return None

        sopa = BeautifulSoup(respuesta.text, 'html.parser')

        # Buscar enlaces de títulos
        resultados = sopa.select('li.ipc-metadata-list-summary-item a[href^="/title/tt"]')

        for enlace in resultados:
            texto_titulo = enlace.get_text(strip=True)
            href = enlace['href']

            # Extraer año si está presente
            li_padre = enlace.find_parent('li')
            coincidencia_año = re.search(r'\d{4}', li_padre.get_text() if li_padre else '')
            año_encontrado = int(coincidencia_año.group()) if coincidencia_año else None

            # Validar coincidencia del año
            if año_encontrado and abs(año_encontrado - int(año_pelicula)) <= 1:
                return f"https://www.imdb.com{href}"

        print(f"No se encontró una coincidencia exacta para {titulo_pelicula} ({año_pelicula})")
        return None

    except Exception as e:
        print(f"Error al buscar en IMDB: {str(e)}")
        return None


def obtener_puntuacion_imdb(url_imdb):
    """
    Extrae la puntuación de IMDB de la página de una película
    """
    cabeceras = {
        'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36'
    }

    print(f"Extrayendo puntuación de IMDB: {url_imdb}")

    try:
        respuesta = requests.get(url_imdb, headers=cabeceras)

        if respuesta.status_code != 200:
            print(f"Error al acceder a {url_imdb}: Código {respuesta.status_code}")
            return None, None

        sopa = BeautifulSoup(respuesta.text, 'html.parser')

        # Buscar puntuación en elementos específicos
        elemento_puntuacion = sopa.select_one('[data-testid="hero-rating-bar__aggregate-rating__score"] span')
        elemento_votos = sopa.select_one('.sc-d541859f-3')

        if elemento_puntuacion:
            puntuacion = elemento_puntuacion.text.strip()
            try:
                puntuacion = float(puntuacion)
            except ValueError:
                puntuacion = None
        else:
            puntuacion = None


        if elemento_votos:
            texto_votos = elemento_votos.text.strip()

            # Convertir votos tipo "224K", "1.2M", etc. a número real
            multiplicador = 1
            if 'K' in texto_votos.upper():
                multiplicador = 1_000
                texto_votos = texto_votos.upper().replace('K', '')
            elif 'M' in texto_votos.upper():
                multiplicador = 1_000_000
                texto_votos = texto_votos.upper().replace('M', '')

            try:
                votos = int(float(texto_votos) * multiplicador)
            except ValueError:
                votos = None
        else:
            votos = None

        if not votos:
            coincidencia_votos = re.search(r'"aggregateRating":.*?"ratingCount":\s*(\d+)', respuesta.text)
            if coincidencia_votos:
                try:
                    votos = int(coincidencia_votos.group(1))
                except ValueError:
                    pass

        return puntuacion, votos

    except Exception as e:
        print(f"Error al extraer puntuación de IMDB: {str(e)}")
        return None, None

def añadir_puntuaciones_imdb_a_excel(ruta_archivo_entrada, ruta_archivo_salida):
    """
    Lee un archivo Excel, busca las películas en IMDB y añade las puntuaciones
    """
    try:
        # Comprobar si el archivo existe
        if not os.path.exists(ruta_archivo_entrada):
            print(f"El archivo {ruta_archivo_entrada} no existe")
            return False

        # Leer el archivo Excel
        print(f"Leyendo archivo Excel: {ruta_archivo_entrada}")
        df = pd.read_excel(ruta_archivo_entrada)

        # Verificar que el DataFrame tiene datos
        if df.empty:
            print("El archivo Excel no contiene datos")
            return False

        print(f"Se han encontrado {len(df)} películas en el Excel")

        # Añadir columnas para puntuación de IMDB y votos
        df['imdb_url'] = None
        df['imdb_puntuacion'] = None
        df['imdb_votos'] = None

        # Para cada película, buscar en IMDB y obtener puntuación
        for indice, fila in df.iterrows():
            titulo = fila['titulo_original']
            año = fila['anio']

            print(f"\nProcesando película {indice+1}/{len(df)}: {titulo} ({año})")

            # Buscar la película en IMDB
            url_imdb = buscar_imdb(titulo, año)

            if url_imdb:
                # Obtener puntuación de IMDB
                puntuacion_imdb, votos_imdb = obtener_puntuacion_imdb(url_imdb)

                # Actualizar DataFrame
                df.at[indice, 'imdb_url'] = url_imdb
                df.at[indice, 'imdb_puntuacion'] = puntuacion_imdb
                df.at[indice, 'imdb_votos'] = votos_imdb

                print(f"Puntuación IMDB para {titulo}: {puntuacion_imdb} ({votos_imdb} votos)")
            else:
                print(f"No se encontró la película en IMDB: {titulo}")

            # Pausa para evitar sobrecarga
            time.sleep(random.uniform(2, 5))

        # Guardar el resultado final
        df.to_excel(ruta_archivo_salida, index=False)
        ruta_csv = ruta_archivo_salida.replace('.xlsx', '.csv')
        df.to_csv(ruta_csv, sep=';', index=False, encoding='utf-8-sig')
        print(f"Archivo CSV exportado a: {ruta_csv}")
        print(f"\nProceso completado. Archivo guardado en: {ruta_archivo_salida}")

        # Estadísticas
        contador_imdb = df['imdb_puntuacion'].count()
        print(f"Se encontraron puntuaciones de IMDB para {contador_imdb} de {len(df)} películas ({contador_imdb/len(df)*100:.1f}%)")

        if contador_imdb > 0:
            puntuacion_media = df['imdb_puntuacion'].mean()
            print(f"Puntuación media de IMDB: {puntuacion_media:.1f}")

        return True

    except Exception as e:
        print(f"Error al procesar el archivo Excel: {str(e)}")
        return False

def obtener_archivos_disponibles():
    """
    Obtiene la lista de archivos Excel disponibles en la carpeta de scraping
    """
    ruta_base = "/content/drive/My Drive/Tipología/PEC2/scraping/"

    # Buscar todos los archivos Excel que coincidan con el patrón de nombre de archivo
    patron = ruta_base + "peliculas_filmaffinity_*.xlsx"
    archivos = glob.glob(patron)

    # Excluir archivos que ya contengan "imdb" en el nombre
    archivos = [archivo for archivo in archivos if "imdb" not in archivo]

    return archivos

if __name__ == "__main__":
    # Obtener archivos disponibles
    archivos_disponibles = obtener_archivos_disponibles()

    if not archivos_disponibles:
        print("No se encontraron archivos de películas en la carpeta.")
        exit(1)

    print("=== AÑADIR PUNTUACIONES DE IMDB A EXCEL ===")
    print("Archivos disponibles:")

    for i, archivo in enumerate(archivos_disponibles, 1):
        # Extraer el nombre de archivo sin la ruta
        nombre_archivo = os.path.basename(archivo)

        # Intentar extraer el rango de años del nombre de archivo
        coincidencia = re.search(r'(\d{4})-(\d{4})', nombre_archivo)
        if coincidencia:
            anio_inicio, anio_fin = coincidencia.groups()
            print(f"{i}. {nombre_archivo} (Rango de años: {anio_inicio}-{anio_fin})")
        else:
            print(f"{i}. {nombre_archivo}")

    # Solicitar al usuario que seleccione un archivo
    seleccion = 0
    while seleccion < 1 or seleccion > len(archivos_disponibles):
        try:
            seleccion = int(input(f"\nSeleccione un archivo (1-{len(archivos_disponibles)}): "))
        except ValueError:
            print("Por favor, ingrese un número válido.")

    archivo_entrada = archivos_disponibles[seleccion - 1]

    # Generar el nombre del archivo de salida
    nombre_base = os.path.basename(archivo_entrada)
    nombre_sin_extension = os.path.splitext(nombre_base)[0]

    # Crear nombre para el archivo de salida
    if "prueba" in nombre_base:
        archivo_salida = f"/content/drive/My Drive/Tipología/PEC2/scraping/{nombre_sin_extension}_imdb_prueba.xlsx"
    else:
        archivo_salida = f"/content/drive/My Drive/Tipología/PEC2/scraping/{nombre_sin_extension}_imdb.xlsx"

    print(f"\nArchivo de entrada: {archivo_entrada}")
    print(f"Archivo de salida: {archivo_salida}")

    # Ejecutar el proceso
    añadir_puntuaciones_imdb_a_excel(archivo_entrada, archivo_salida)

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
=== AÑADIR PUNTUACIONES DE IMDB A EXCEL ===
Archivos disponibles:
1. peliculas_filmaffinity_1980-1989.xlsx (Rango de años: 1980-1989)
2. peliculas_filmaffinity_1990-1999.xlsx (Rango de años: 1990-1999)

Seleccione un archivo (1-2): 2

Archivo de entrada: /content/drive/My Drive/Tipología/PEC2/scraping/peliculas_filmaffinity_1990-1999.xlsx
Archivo de salida: /content/drive/My Drive/Tipología/PEC2/scraping/peliculas_filmaffinity_1990-1999_imdb.xlsx
Leyendo archivo Excel: /content/drive/My Drive/Tipología/PEC2/scraping/peliculas_filmaffinity_1990-1999.xlsx
Se han encontrado 40 películas en el Excel

Procesando película 1/40: Pretty Woman (1990)
Buscando en IMDB: Pretty Woman (1990)
Extrayendo puntuación de IMDB: https://www.imdb.com/title/tt0100405/?ref_=fn_ttl_ttl_1
Puntuación IMDB para Pretty Woman: 7.1 (384000 votos)

Procesando película 2/40: Home Alone (199